In [1]:
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Dense, Conv2D, Input, MaxPool2D, UpSampling2D, Concatenate, Conv2DTranspose
import tensorflow as tf
from keras.optimizers import Adam
from scipy.misc import imresize
from scipy.misc import imsave
import imageio
from tqdm import tqdm
import os
from keras.preprocessing.image import array_to_img, img_to_array, load_img, ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

import re
import random
from PIL import Image

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


###### Set the random seed

In [2]:
random.seed(100)
np.random.seed(100)

###### Define the train image and mask directory

In [3]:
image_data_folder = '../data_road/training/image_2/'
mask_data_folder = '../data_road/training/gt_image_2/'

###### List of training images and their masks

In [4]:
all_images = os.listdir(image_data_folder)
all_images.sort()
all_images = np.array(all_images)

all_mask_images = {}
for image_file in all_images:
    all_mask_images[image_file] = re.sub('_', '_road_',image_file)

###### Read test images

In [5]:
test_dir = "../data_road/testing/image_2/"
test_images = os.listdir(test_dir)
test_images.sort()
test_images = np.array(test_images)

###### Directory where predictions are written

In [6]:
test_predict_dir = 'test_predict/'

###### Image size and batch size

In [7]:
width = 1280
height = 384
batch_size = 2

###### Split data for training and validation

In [8]:
train_images = all_images[:232]
valid_images = all_images[232:]

steps_per_epoch = (train_images.shape[0])/batch_size
validation_steps = valid_images.shape[0]/batch_size

###### Batch generator used in fit_generator 

In [9]:
def batch_generator(images, batch_size):
        """
        data_dir: where the actual images are kept
        mask_dir: where the actual masks are kept
        images: the filenames of the images we want to generate batches from
        batch_size: self explanatory
        dims: the dimensions in which we want to rescale our images
        """
        while True:
            ix = np.random.choice(np.arange(images.shape[0]), batch_size)
            batch_features = np.zeros((batch_size, height, width,3), dtype='float32')
            batch_labels = np.zeros((batch_size, height, width, 1), dtype='float32')

            index = 0
            for i in ix:
                if ( np.random.uniform(0,1)>5):
                    flip = 1
                else:
                    flip = 0
                # images
                original_img = load_img(image_data_folder + images[i])
                resized_img = imresize(original_img, [height, width, 3])                
                img_array = img_to_array(resized_img)/255.
                
                batch_features[index,:,:,:] = img_array
                
                # masks
                original_mask = load_img(mask_data_folder + all_mask_images[images[i]])
                resized_mask = imresize(original_mask, [height, width, 3])
                label = img_to_array(resized_mask)/255.
                batch_labels[index,:,:,0] = label[:,:,2]
                    
                index = index + 1
                                
            yield batch_features, batch_labels

###### Define downscaling and upscaling filters

In [10]:
def down(input_layer, filters, pool=True):
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(input_layer)
    residual = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    if pool:
        max_pool = MaxPool2D()(residual)
        return max_pool, residual
    else:
        return residual

def up(input_layer, residual, filters):
    filters=int(filters)
    upsample = UpSampling2D()(input_layer)
    upconv = Conv2D(filters, kernel_size=(2, 2), padding="same")(upsample)
    concat = Concatenate(axis=3)([residual, upconv])
    conv1 = Conv2D(filters, (3, 3), padding='same', activation='relu')(concat)
    conv2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(conv1)
    return conv2

###### Define custom U-net

In [11]:
filters = 64
input_layer = Input(shape = [height, width, 3])
layers = [input_layer]
residuals = []

# Down 1, 1280, 384
d1, res1 = down(input_layer, filters)
residuals.append(res1)

# Down 2, 640, 192
filters *= 2
d2, res2 = down(d1, filters)
residuals.append(res2)

# Down 3, 320, 96
filters *= 2
d3, res3 = down(d2, filters)
residuals.append(res3)

# Down 4, 160, 48
filters += 32
d4, res4 = down(d3, filters)
residuals.append(res4)

# Down 5, 80, 24
d5, res5 = down(d4, filters)
residuals.append(res5)

# Down 6, 40, 12
d6, res6 = down(d5, filters)
residuals.append(res6)

# Down 7, 20, 6
d7, res7 = down(d6, filters)
residuals.append(res7)

# Down 8, 10, 3
d8 = down(d7, filters, pool=False)

# Up 1
up1 = up(d8, residual=residuals[-1], filters=filters/2)

# Up 2 
up2 = up(up1, residual=residuals[-2], filters=filters/2)

# Up 3 
up3 = up(up2, residual=residuals[-3], filters=filters/2)

# Up 4 
up4 = up(up3, residual=residuals[-4], filters=filters/2)

# Up 5 
up5 = up(up4, residual=residuals[-5], filters=filters/2)
filters /= 2

# Up 6 
up6 = up(up5, residual=residuals[-6], filters=filters/2)
filters /= 2

# Up 7 
up7 = up(up6, residual=residuals[-7], filters=filters/2)
filters /= 2

out = Conv2D(filters=1, kernel_size=(1, 1), activation="sigmoid")(up7)

model = Model(input_layer, out)

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 384, 1280, 3)  0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 384, 1280, 64) 1792        input_1[0][0]                    
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 384, 1280, 64) 36928       conv2d_1[0][0]                   
____________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)   (None, 192, 640, 64)  0           conv2d_2[0][0]                   
___________________________________________________________________________________________

###### Define dice_coefficient

In [12]:
def dice_coef(y_true, y_pred):
    smooth = 1e-5
    
    y_true = tf.round(tf.reshape(y_true, [-1]))
    y_pred = tf.round(tf.reshape(y_pred, [-1]))
    
    isct = tf.reduce_sum(y_true * y_pred)
    
    return 2 * isct / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred))

###### Training

In [13]:
kfold_weights_path = 'weights.h5'

callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)]

In [14]:
model.compile(optimizer=Adam(5e-4), loss='binary_crossentropy', metrics=[dice_coef])
epochs=1
model.fit_generator(batch_generator(train_images, batch_size), steps_per_epoch, epochs=epochs, verbose=1,
                        callbacks=callbacks, validation_data=batch_generator(valid_images, batch_size),
                        validation_steps=validation_steps)

Epoch 1/1
116/116 [==============================] - 68s - loss: 0.3147 - dice_coef: 0.2440 - val_loss: 0.3383 - val_dice_coef: 0.7027


In [15]:
model.compile(optimizer=Adam(2e-4), loss='binary_crossentropy', metrics=[dice_coef])
epochs=1
model.fit_generator(batch_generator(train_images, batch_size), steps_per_epoch, epochs=epochs, verbose=1,
                        callbacks=callbacks, validation_data=batch_generator(valid_images, batch_size),
                        validation_steps=validation_steps)

Epoch 1/1
116/116 [==============================] - 58s - loss: 0.1136 - dice_coef: 0.8821 - val_loss: 0.2390 - val_dice_coef: 0.7267


In [16]:
model.compile(optimizer=Adam(1e-4), loss='binary_crossentropy', metrics=[dice_coef])
epochs=10
model.fit_generator(batch_generator(train_images, batch_size), steps_per_epoch, epochs=epochs, verbose=1,
                        callbacks=callbacks, validation_data=batch_generator(valid_images, batch_size),
                        validation_steps=validation_steps)

Epoch 1/10
116/116 [==============================] - 58s - loss: 0.0987 - dice_coef: 0.8928 - val_loss: 0.1158 - val_dice_coef: 0.8289
Epoch 2/10
116/116 [==============================] - 56s - loss: 0.0864 - dice_coef: 0.9019 - val_loss: 0.1226 - val_dice_coef: 0.8089
Epoch 3/10
116/116 [==============================] - 56s - loss: 0.0816 - dice_coef: 0.9095 - val_loss: 0.1327 - val_dice_coef: 0.7928
Epoch 4/10
116/116 [==============================] - 56s - loss: 0.0797 - dice_coef: 0.9113 - val_loss: 0.1263 - val_dice_coef: 0.8183
Epoch 5/10
116/116 [==============================] - 56s - loss: 0.0606 - dice_coef: 0.9350 - val_loss: 0.0928 - val_dice_coef: 0.8495
Epoch 6/10
116/116 [==============================] - 55s - loss: 0.0772 - dice_coef: 0.9186 - val_loss: 0.0979 - val_dice_coef: 0.8437
Epoch 7/10
116/116 [==============================] - 55s - loss: 0.0681 - dice_coef: 0.9349 - val_loss: 0.1074 - val_dice_coef: 0.8764
Epoch 8/10
116/116 [============================

In [17]:
model.compile(optimizer=Adam(5e-5), loss='binary_crossentropy', metrics=[dice_coef])
epochs=5
model.fit_generator(batch_generator(train_images, batch_size), steps_per_epoch, epochs=epochs, verbose=1,
                        callbacks=callbacks, validation_data=batch_generator(valid_images, batch_size),
                        validation_steps=validation_steps)

Epoch 1/5
116/116 [==============================] - 58s - loss: 0.0389 - dice_coef: 0.9591 - val_loss: 0.0837 - val_dice_coef: 0.8864
Epoch 2/5
116/116 [==============================] - 55s - loss: 0.0350 - dice_coef: 0.9623 - val_loss: 0.0891 - val_dice_coef: 0.8930
Epoch 3/5
116/116 [==============================] - 55s - loss: 0.0341 - dice_coef: 0.9627 - val_loss: 0.0697 - val_dice_coef: 0.9103
Epoch 4/5
116/116 [==============================] - 55s - loss: 0.0291 - dice_coef: 0.9689 - val_loss: 0.0993 - val_dice_coef: 0.8888
Epoch 5/5
116/116 [==============================] - 55s - loss: 0.0324 - dice_coef: 0.9647 - val_loss: 0.0833 - val_dice_coef: 0.9066


In [18]:
model.compile(optimizer=Adam(1e-5), loss='binary_crossentropy', metrics=[dice_coef])
epochs=5
model.fit_generator(batch_generator(train_images, batch_size), steps_per_epoch, epochs=epochs, verbose=1,
                        callbacks=callbacks, validation_data=batch_generator(valid_images, batch_size),
                        validation_steps=validation_steps)

Epoch 1/5
116/116 [==============================] - 58s - loss: 0.0260 - dice_coef: 0.9704 - val_loss: 0.0768 - val_dice_coef: 0.9145
Epoch 2/5
116/116 [==============================] - 55s - loss: 0.0239 - dice_coef: 0.9735 - val_loss: 0.0846 - val_dice_coef: 0.9113
Epoch 3/5
116/116 [==============================] - 55s - loss: 0.0246 - dice_coef: 0.9727 - val_loss: 0.0870 - val_dice_coef: 0.9066
Epoch 4/5
116/116 [==============================] - 55s - loss: 0.0232 - dice_coef: 0.9737 - val_loss: 0.0963 - val_dice_coef: 0.9061
Epoch 5/5
116/116 [==============================] - 55s - loss: 0.0221 - dice_coef: 0.9749 - val_loss: 0.0978 - val_dice_coef: 0.9176


In [19]:
model.compile(optimizer=Adam(5e-6), loss='binary_crossentropy', metrics=[dice_coef])
epochs=5
model.fit_generator(batch_generator(train_images, batch_size), steps_per_epoch, epochs=epochs, verbose=1,
                        callbacks=callbacks, validation_data=batch_generator(valid_images, batch_size),
                        validation_steps=validation_steps)

Epoch 1/5
116/116 [==============================] - 57s - loss: 0.0217 - dice_coef: 0.9754 - val_loss: 0.0924 - val_dice_coef: 0.9077
Epoch 2/5
116/116 [==============================] - 54s - loss: 0.0221 - dice_coef: 0.9754 - val_loss: 0.1010 - val_dice_coef: 0.9129
Epoch 3/5
116/116 [==============================] - 55s - loss: 0.0221 - dice_coef: 0.9749 - val_loss: 0.1148 - val_dice_coef: 0.9093
Epoch 4/5
116/116 [==============================] - 55s - loss: 0.0215 - dice_coef: 0.9760 - val_loss: 0.0933 - val_dice_coef: 0.9182
Epoch 5/5
116/116 [==============================] - 54s - loss: 0.0216 - dice_coef: 0.9750 - val_loss: 0.1004 - val_dice_coef: 0.9124


In [20]:
model.compile(optimizer=Adam(5e-6), loss='binary_crossentropy', metrics=[dice_coef])
epochs=10
model.fit_generator(batch_generator(train_images, batch_size), steps_per_epoch, epochs=epochs, verbose=1,
                        callbacks=callbacks, validation_data=batch_generator(valid_images, batch_size),
                        validation_steps=validation_steps)

Epoch 1/10
116/116 [==============================] - 57s - loss: 0.0215 - dice_coef: 0.9760 - val_loss: 0.1188 - val_dice_coef: 0.9036
Epoch 2/10
116/116 [==============================] - 54s - loss: 0.0209 - dice_coef: 0.9764 - val_loss: 0.1056 - val_dice_coef: 0.9153
Epoch 3/10
116/116 [==============================] - 54s - loss: 0.0199 - dice_coef: 0.9764 - val_loss: 0.1168 - val_dice_coef: 0.9036
Epoch 4/10
116/116 [==============================] - 54s - loss: 0.0197 - dice_coef: 0.9773 - val_loss: 0.0808 - val_dice_coef: 0.9240
Epoch 5/10
116/116 [==============================] - 55s - loss: 0.0200 - dice_coef: 0.9770 - val_loss: 0.1328 - val_dice_coef: 0.9084
Epoch 6/10
116/116 [==============================] - 54s - loss: 0.0193 - dice_coef: 0.9779 - val_loss: 0.1017 - val_dice_coef: 0.9133
Epoch 7/10
116/116 [==============================] - 55s - loss: 0.0187 - dice_coef: 0.9790 - val_loss: 0.1199 - val_dice_coef: 0.9101
Epoch 8/10
116/116 [============================

In [21]:
model.save_weights('weights.h5')

###### Write just the predicted mask (road)

In [22]:
def write_to_pngfiles(batch_labels, images, istart, iend):

    for i in range(istart,iend):
        img_array = batch_labels[i-istart,:,:,0]*255
        
        img = Image.fromarray((img_array).astype(np.uint8))
        img_path = test_predict_dir + images[i][:-4] + '.png'
        img.save(img_path)
        return 

###### Write the predicted mask (road) superimposed on the original image

In [23]:
def write_to_jpgfiles(batch_features, batch_labels, images, istart, iend):

    for i in range(istart,iend):
        img_array = batch_features[i-istart,:,:,:]
        img_array[:,:,0] = img_array[:,:,0] + batch_labels[i-istart,:,:,0]
        img_array = img_array*255
        
        img_path = test_predict_dir + images[i][:-4] + '.jpg'
        imsave(img_path, img_array)
        return 

In [24]:
def test_prediction(model, images, batch_size=1):
    
    count = 0
    total_images = images.shape[0]
    total_steps = int(total_images/batch_size)
    print(total_images, batch_size, total_steps)
    
    batch_features = np.zeros((batch_size, height, width,3), dtype='float32')
    batch_labels = np.zeros((batch_size, height, width, 1), np.float32)
    
    for steps in range(total_steps):
        ishift = steps*batch_size
        for i in range(batch_size):
            img_path = test_dir + images[count]
                    
            original_img = load_img(img_path)
            resized_img = imresize(original_img, [height, width, 3])
            img_array = img_to_array(resized_img)/255.            
            batch_features[i,:,:,:] = img_array
            count = count + 1        
            
        batch_labels  = model.predict(batch_features, batch_size = batch_size, verbose=0)
        write_to_pngfiles(batch_labels, images, ishift, ishift+batch_size)
        write_to_jpgfiles(batch_features, batch_labels, images, ishift, ishift+batch_size)

    if (count < total_images):
        istart = count
        iend = total_images

        for i in range(istart-count, iend-count):
            
            img_path = test_dir + images[count]
            
            original_img = load_img(img_path)
            resized_img = resize(original_img, [height_orig, width_orig, 3])
            img_array = img_to_array(resized_img)/255.   
            batch_features[i,:,:,:] = img_array
            count = count + 1
                       
        batch_labels  = model.predict(batch_features, batch_size = batch_size, verbose=0)            
        write_to_pngfiles(batch_labels, images, istart, iend)
        write_to_jpgfiles(batch_features, batch_labels, images, istart, iend)
                
    return


In [25]:
test_prediction(model, test_images, batch_size=1)

290 1 290
